In [1]:
import os
import imgaug
from skimage import io, transform
from skimage import img_as_ubyte
from imgaug import augmenters as iaa
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
import random

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Flayiva_01\anaconda3\envs\pytorchenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
BASE_LABELS = "data/AnnotationsByMD/400_senior"
TEST_BASE_LABELS = "data/Test/AnnotationsByMD"

In [3]:
BASE_PATH = "data/images/raw/256/train"

In [4]:
def img_path_walker(base_path):
    # walk through subdirs to get all image paths
    img_paths = []
    for root, _, files in os.walk(base_path):
        for name in files:
            if name.endswith(".bmp"):
                img_paths.append(os.path.abspath(os.path.join(root, name)))
    return img_paths

In [5]:
img_paths = img_path_walker(BASE_PATH)

In [6]:
def extract_labels_from_txt(path):
    with open(path, "r") as f:
        # only first 19 are actual coords in dataset label files
        coords_raw = f.readlines()[:19]
        coords_raw = [tuple([int(float(s)) for s in t.split(",")]) for t in coords_raw]
        return coords_raw

In [7]:
def label_walker(base_labels):
    img_labels = {}
    for label_file in os.listdir(base_labels):
        linked_img = label_file[:-3] + "bmp"
        img_labels[linked_img] = extract_labels_from_txt(base_labels + "/" + label_file)
    return img_labels

In [8]:
img_labels = label_walker(BASE_LABELS)

In [9]:
len(img_labels)

400

In [10]:
len(img_paths)

150

In [11]:
SAMPLE_PATH = "data/Test/image/001.bmp"
img = img_as_ubyte(io.imread(SAMPLE_PATH, as_gray=True))
img.shape

(2400, 1935)

In [12]:
kp_to_list = lambda k_arr: [(p.x, p.y) for p in k_arr]

In [13]:
kp_to_list

<function __main__.<lambda>(k_arr)>

In [14]:
def list_to_kp(coords_raw):
    kp = []
    for coord in coords_raw:
        kp.append(imgaug.Keypoint(x=coord[0], y=coord[1]))
    keypoints = imgaug.KeypointsOnImage(kp, shape=img.shape)
    return keypoints

In [15]:
# 1px
augment_functions = [iaa.Affine(translate_px={"y": (18)})]
augment_functions += [iaa.Affine(translate_px={"y": (-18)})]
augment_functions += [iaa.Affine(translate_px={"x": (15)})]
augment_functions += [iaa.Affine(translate_px={"x": (-15)})]

# 2px
augment_functions += [iaa.Affine(translate_px={"y": (37)})]
augment_functions += [iaa.Affine(translate_px={"y": (-37)})]
augment_functions += [iaa.Affine(translate_px={"x": (30)})]
augment_functions += [iaa.Affine(translate_px={"x": (-30)})]

# 3px
augment_functions += [iaa.Affine(translate_px={"y": (56)})]
augment_functions += [iaa.Affine(translate_px={"y": (-56)})]
augment_functions += [iaa.Affine(translate_px={"x": (45)})]
augment_functions += [iaa.Affine(translate_px={"x": (-45)})]

In [16]:
def aug_label_and_save(img, img_name, label, aug_list, base_path):
    kp = [list_to_kp(label)]
    
    print("**************************")
    img_save_name = base_path + "/" + img_name + "_ps_aug0"
   
    with open(img_save_name + ".txt", "w") as lf:
            stringified = [str(tup) for tup in kp_to_list(kp[0].keypoints)]
            print("\n".join(stringified))
            stringified = [s.replace("(", "").replace(")","") for s in stringified]
            
            lf.write("\n".join(stringified))
    for i, aug in enumerate(aug_list):
        
        kp_aug = aug.augment_keypoints(kp)
        img_save_name = base_path + "/" + img_name + "_ps_aug{}".format(i+1)
        # save labelfile:
        with open(img_save_name + ".txt", "w") as lf:
            stringified = [str(tup) for tup in kp_to_list(kp_aug[0].keypoints)]
            stringified = [s.replace("(", "").replace(")","") for s in stringified]
            lf.write("\n".join(stringified))

In [18]:
for i, path in enumerate(img_paths):
    loop_img = io.imread(path)
    
    loop_labels = img_labels[path.split("\\")[-1]]
    aug_label_and_save(loop_img, path.split("\\")[-1][:-4], loop_labels, augment_functions, "data/images/1px_3px/256/train_annots")

**************************
(835, 996)
(1473, 1029)
(1289, 1279)
(604, 1228)
(1375, 1654)
(1386, 2019)
(1333, 2200)
(1263, 2272)
(1305, 2252)
(694, 1805)
(1460, 1870)
(1450, 1864)
(1588, 1753)
(1569, 2013)
(1514, 1620)
(1382, 2310)
(944, 1506)
(1436, 1569)
(664, 1340)
**************************
(810, 1052)
(1395, 985)
(1281, 1238)
(602, 1258)
(1393, 1588)
(1349, 1856)
(1381, 2005)
(1320, 2091)
(1363, 2063)
(687, 1756)
(1407, 1718)
(1426, 1734)
(1519, 1669)
(1512, 1835)
(1462, 1515)
(1459, 2103)
(978, 1433)
(1399, 1445)
(643, 1353)
**************************
(761, 1105)
(1329, 911)
(1250, 1181)
(622, 1279)
(1436, 1415)
(1439, 1744)
(1460, 1882)
(1406, 1935)
(1448, 1920)
(815, 1710)
(1451, 1575)
(1495, 1608)
(1581, 1449)
(1596, 1687)
(1524, 1369)
(1526, 1923)
(1008, 1404)
(1420, 1350)
(712, 1348)
**************************
(877, 1094)
(1536, 897)
(1373, 1239)
(643, 1235)
(1533, 1591)
(1467, 1833)
(1468, 2016)
(1418, 2091)
(1449, 2061)
(788, 1842)
(1489, 1716)
(1537, 1740)
(1655, 1695)
(16

(765, 1007)
(1341, 950)
(1200, 1220)
(592, 1215)
(1304, 1463)
(1322, 1818)
(1323, 1905)
(1261, 1969)
(1302, 1953)
(739, 1694)
(1356, 1638)
(1340, 1645)
(1468, 1551)
(1460, 1751)
(1411, 1458)
(1403, 1997)
(936, 1383)
(1344, 1416)
(662, 1265)
**************************
(748, 1022)
(1323, 1003)
(1183, 1270)
(514, 1201)
(1246, 1516)
(1259, 1894)
(1259, 2028)
(1204, 2078)
(1239, 2063)
(626, 1744)
(1294, 1740)
(1301, 1732)
(1415, 1635)
(1388, 1829)
(1372, 1532)
(1328, 2096)
(878, 1419)
(1297, 1492)
(571, 1240)
**************************
(808, 1025)
(1393, 1050)
(1233, 1299)
(581, 1200)
(1318, 1591)
(1396, 1949)
(1390, 2144)
(1343, 2188)
(1374, 2173)
(737, 1810)
(1438, 1790)
(1371, 1817)
(1491, 1741)
(1538, 1870)
(1401, 1620)
(1470, 2169)
(922, 1449)
(1344, 1544)
(647, 1259)
**************************
(815, 1042)
(1379, 1074)
(1250, 1307)
(569, 1211)
(1291, 1574)
(1247, 1881)
(1220, 2008)
(1152, 2068)
(1193, 2047)
(694, 1756)
(1314, 1749)
(1335, 1772)
(1453, 1731)
(1420, 1857)
(1455, 1603)
(1

(781, 1065)
(1351, 983)
(1237, 1278)
(624, 1287)
(1298, 1566)
(1224, 1851)
(1228, 1979)
(1177, 2025)
(1212, 2010)
(734, 1701)
(1297, 1692)
(1339, 1722)
(1464, 1612)
(1417, 1781)
(1429, 1520)
(1323, 2046)
(929, 1449)
(1333, 1466)
(699, 1327)
**************************
(773, 1020)
(1266, 832)
(1224, 1109)
(611, 1245)
(1381, 1383)
(1300, 1636)
(1291, 1790)
(1262, 1831)
(1287, 1819)
(764, 1710)
(1361, 1524)
(1427, 1541)
(1544, 1453)
(1527, 1633)
(1499, 1337)
(1433, 1869)
(935, 1315)
(1378, 1295)
(699, 1288)
**************************
(823, 1071)
(1422, 904)
(1273, 1173)
(627, 1293)
(1498, 1527)
(1400, 1764)
(1343, 1938)
(1282, 2002)
(1322, 1980)
(756, 1709)
(1479, 1636)
(1569, 1700)
(1675, 1630)
(1597, 1815)
(1615, 1469)
(1428, 1995)
(1009, 1447)
(1502, 1419)
(742, 1372)
**************************
(817, 1113)
(1388, 1058)
(1254, 1281)
(620, 1297)
(1384, 1581)
(1386, 1849)
(1351, 1986)
(1281, 2057)
(1327, 2033)
(739, 1810)
(1431, 1733)
(1453, 1738)
(1549, 1693)
(1545, 1846)
(1486, 1557)
(14

(875, 972)
(1470, 932)
(1313, 1188)
(658, 1220)
(1436, 1511)
(1509, 1883)
(1526, 2016)
(1435, 2111)
(1500, 2076)
(842, 1844)
(1512, 1715)
(1505, 1686)
(1658, 1648)
(1659, 1789)
(1551, 1500)
(1618, 2091)
(1055, 1404)
(1436, 1441)
(729, 1349)
**************************
(813, 1039)
(1397, 915)
(1279, 1193)
(656, 1200)
(1384, 1494)
(1331, 1782)
(1373, 1959)
(1334, 2045)
(1369, 2010)
(744, 1840)
(1378, 1622)
(1462, 1685)
(1569, 1576)
(1509, 1743)
(1514, 1463)
(1435, 2025)
(989, 1460)
(1405, 1408)
(709, 1389)
**************************
(795, 1044)
(1402, 908)
(1276, 1172)
(648, 1205)
(1455, 1471)
(1409, 1810)
(1407, 1958)
(1342, 2027)
(1389, 2007)
(770, 1792)
(1483, 1628)
(1548, 1655)
(1639, 1566)
(1608, 1756)
(1580, 1441)
(1483, 1991)
(1003, 1427)
(1460, 1394)
(675, 1371)
**************************
(840, 1149)
(1369, 1110)
(1279, 1355)
(674, 1216)
(1329, 1601)
(1262, 1882)
(1215, 2013)
(1157, 2052)
(1192, 2046)
(764, 1722)
(1328, 1761)
(1356, 1775)
(1462, 1668)
(1433, 1879)
(1398, 1570)
(13

(880, 1079)
(1442, 1076)
(1325, 1339)
(640, 1218)
(1373, 1609)
(1331, 1912)
(1328, 2036)
(1275, 2089)
(1306, 2063)
(648, 1792)
(1365, 1773)
(1390, 1793)
(1523, 1723)
(1489, 1875)
(1484, 1616)
(1421, 2090)
(979, 1505)
(1397, 1558)
(669, 1375)
**************************
(803, 973)
(1379, 950)
(1218, 1207)
(595, 1174)
(1314, 1415)
(1289, 1745)
(1289, 1824)
(1234, 1899)
(1275, 1874)
(669, 1637)
(1339, 1593)
(1396, 1609)
(1488, 1539)
(1445, 1717)
(1457, 1415)
(1360, 1925)
(946, 1368)
(1374, 1369)
(655, 1357)
**************************
(785, 957)
(1356, 934)
(1229, 1180)
(606, 1147)
(1383, 1450)
(1400, 1792)
(1434, 1938)
(1396, 2009)
(1426, 1987)
(760, 1724)
(1405, 1644)
(1422, 1646)
(1546, 1585)
(1556, 1733)
(1481, 1479)
(1545, 2021)
(965, 1361)
(1370, 1424)
(668, 1273)
**************************
(832, 1004)
(1439, 947)
(1302, 1207)
(615, 1205)
(1420, 1584)
(1420, 1877)
(1413, 2055)
(1360, 2131)
(1399, 2102)
(768, 1870)
(1443, 1731)
(1453, 1726)
(1583, 1646)
(1586, 1830)
(1526, 1531)
(1504,